# Azure Files Demo

## Preparation

Install the NuGet package

In [28]:
#r "nuget:Azure.Storage.Files.Shares"

Installed Packages Azure.Storage.Files.Shares, 12.23.0


Include namespaces:

In [29]:
using System;
using System.IO;
using Azure;
using Azure.Storage.Files.Shares;
using Azure.Storage.Files.Shares.Models;
using Azure.Storage.Sas;

Define some constants we would need later:

In [ ]:
const string AzureConnectionString = "DefaultEndpointsProtocol=https;AccountName=ztechtest;AccountKey=XXX;EndpointSuffix=core.windows.net";
static string ShareName = "myshare" + Guid.NewGuid().ToString()[..8];

Console.WriteLine($"Using share name: {ShareName}");

Using share name: myshare132dbab5


## Open/create the file share

In [31]:
// Create a ShareClient
var share = new ShareClient(AzureConnectionString, ShareName);

// Create the share if it doesn't exist
share.CreateIfNotExists();

Console.WriteLine($"Share '{ShareName}' is ready.");

Share 'myshare132dbab5' is ready.


## Work with data

Upload some data from local computer:

In [32]:
var directory = share.GetRootDirectoryClient();
var localDirectoryPath = @"C:\Windows\Web\Wallpaper\ThemeA";
var localJpgFiles = Directory.GetFiles(localDirectoryPath , "*.jpg");
foreach (var filePath in localJpgFiles) {
    var fileName = Path.GetFileName(filePath);

    // Open local file for reading
    using var localFileStream = File.OpenRead(filePath);

    // Create remote file for writing
    var fileClient = directory.GetFileClient(fileName);
    await fileClient.DeleteIfExistsAsync();
    await fileClient.CreateAsync(localFileStream.Length);

    // Upload file content
    await fileClient.UploadRangeAsync(new HttpRange(0, localFileStream.Length), localFileStream);
    fileClient.Uri.ToString().Display();
}

https://ztechtest.file.core.windows.net/myshare132dbab5/img20.jpg

https://ztechtest.file.core.windows.net/myshare132dbab5/img21.jpg

https://ztechtest.file.core.windows.net/myshare132dbab5/img22.jpg

https://ztechtest.file.core.windows.net/myshare132dbab5/img23.jpg

In addition to using REST API, we can use SMB to connect to the remote file system:

* **Path:** `\\<name>.file.core.windows.net\<share>`
* **User name:** `<name>`
* **Password:** _primary or secondary access key_

Based on settings, we can also use NFS protocol and various other forms of authentication, such as the Entra ID.

Print all remote files:

In [33]:
var remoteItems = directory.GetFilesAndDirectoriesAsync();
await foreach (var item in remoteItems) {
    item.Display();
}

Azure.Storage.Files.Shares.Models.ShareFileItem IsDirectory False Name img20.jpg Id 13835128424026341376 Properties Azure.Storage.Files.Shares.Models.ShareFileItemProperties CreatedOn <null> LastAccessedOn <null> LastWrittenOn <null> ChangedOn <null> LastModified <null> ETag <null> FileAttributes <null> PermissionKey <null> FileSize 826734

Azure.Storage.Files.Shares.Models.ShareFileItem IsDirectory False Name img21.jpg Id 11529285414812647424 Properties Azure.Storage.Files.Shares.Models.ShareFileItemProperties CreatedOn <null> LastAccessedOn <null> LastWrittenOn <null> ChangedOn <null> LastModified <null> ETag <null> FileAttributes <null> PermissionKey <null> FileSize 833851

Azure.Storage.Files.Shares.Models.ShareFileItem IsDirectory False Name img22.jpg Id 16140971433240035328 Properties Azure.Storage.Files.Shares.Models.ShareFileItemProperties CreatedOn <null> LastAccessedOn <null> LastWrittenOn <null> ChangedOn <null> LastModified <null> ETag <null> FileAttributes <null> PermissionKey <null> FileSize 760748

Azure.Storage.Files.Shares.Models.ShareFileItem IsDirectory False Name img23.jpg Id 10376363910205800448 Properties Azure.Storage.Files.Shares.Models.ShareFileItemProperties CreatedOn <null> LastAccessedOn <null> LastWrittenOn <null> ChangedOn <null> LastModified <null> ETag <null> FileAttributes <null> PermissionKey <null> FileSize 907112

Download all the remote files to temporary folder:

In [34]:
// Get path to temporary directory
var tempDirectory = Path.GetTempPath();

// Process all remote files and download them to temporary folder
await foreach (var item in remoteItems) {
    var localFilePath = Path.Combine(tempDirectory, item.Name);
    
    // Create file client
    var fileClient = directory.GetFileClient(item.Name);

    // Download the file
    var result = await fileClient.DownloadAsync();
    if(result.Value == null) {
        Console.WriteLine($"Failed to download file {item.Name}");
        continue;
    }

    // Save the file to the local file system
    using (var stream = File.OpenWrite(localFilePath)) {
        await result.Value.Content.CopyToAsync(stream);
    }
    localFilePath.Display();
}

C:\Users\Altair\AppData\Local\Temp\img20.jpg

C:\Users\Altair\AppData\Local\Temp\img21.jpg

C:\Users\Altair\AppData\Local\Temp\img22.jpg

C:\Users\Altair\AppData\Local\Temp\img23.jpg

## Cleanup

Delete the share including all files:

In [35]:
share.DeleteIfExists();